In [1]:
import cv2
import imutils
import pytesseract
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk

# Set the path to the Tesseract OCR engine executable
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract'

# Create global variables to store image paths
car_image_path = None
plate_image_path = None

# Create a function to open a file dialog and select an image
def open_image():
    global car_image_path  # Declare car_image_path as global
    file_path = filedialog.askopenfilename()
    if file_path:
        car_image_path = file_path
        display_image(car_image_path)  # Display the car image
        process_image(car_image_path)

# Create a function to process the selected image and display the result
def process_image(image_path):
    image = cv2.imread(image_path)
    image = imutils.resize(image, width=300)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply bilateral filtering for noise reduction
    gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17)

    # Perform edge detection using the Canny algorithm
    edged = cv2.Canny(gray_image, 30, 200)

    # Find contours in the edged image
    cnts, new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area and select the top 30
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:30]

    screenCnt = None
    i = 7
    # Iterate over the contours
    for c in cnts:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
        if len(approx) == 4:
            screenCnt = approx
            x, y, w, h = cv2.boundingRect(c)
            new_img = image[y:y + h, x:x + w]
            cv2.imwrite('./' + str(i) + '.png', new_img)
            plate_image_path = './' + str(i) + '.png'  # Store the plate image path
            break

    # Draw the detected license plate contour on the image
    if screenCnt is not None:
        cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 3)
        cv2.imshow("image with detected license plate", image)
        cv2.waitKey(0)
    else:
        print("No license plate contour found.")
    
    if plate_image_path:
        display_plate_image(plate_image_path)  # Display the plate image
        plate = pytesseract.image_to_string(plate_image_path, lang='eng')
        result_label.config(text="Number plate is: " + plate)
    else:
        print("No plate image found.")

def display_plate_image(image_path):
    image = Image.open(image_path)
    image.thumbnail((300, 300))  # Resize the image to fit in the window
    photo = ImageTk.PhotoImage(image)
    plate_image_label.config(image=photo)
    plate_image_label.image = photo

def display_image(image_path):
    image = Image.open(image_path)
    image.thumbnail((300, 300))  # Resize the image to fit in the window
    photo = ImageTk.PhotoImage(image)
    image_label.config(image=photo)
    image_label.image = photo

# Create a Tkinter window with an "Upload" button
root = tk.Tk()
root.title("Car Name Plate Detection System")

# Create a style for the ttk widgets
style = ttk.Style()
style.configure("TButton", foreground="black", background="green")

# Create a template frame
template_frame = ttk.Frame(root)
template_frame.pack(padx=20, pady=20)

# Create a heading label with bold and underline styling
heading_label = ttk.Label(
    template_frame,
    text="Car Name Plate Detection System",
    font=("Arial", 16, "bold underline")  # Use "bold underline" to specify both bold and underline
)
heading_label.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Create an "Upload" button in the template frame
upload_button = ttk.Button(template_frame, text="Upload Image", command=open_image)
upload_button.grid(row=1, column=0, padx=10, pady=10)

# Create a label to display the result
result_label = ttk.Label(template_frame, text="", font=("Arial", 12))
result_label.grid(row=2, column=0, padx=10, pady=10)

# Create a label to display the car image
image_label = ttk.Label(template_frame)
image_label.grid(row=3, column=0, padx=10, pady=10)

# Create a label to display the plate image
plate_image_label = ttk.Label(template_frame)
plate_image_label.grid(row=4, column=0, padx=10, pady=10)

root.mainloop()